Loading libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind

Load the datasets

In [9]:
df = pd.read_csv('../data/MachineLearningRating_v3.csv')
df.info()
df.head()

/tmp/ipykernel_18428/3613318284.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [13]:
print(df['Gender'].tail())

1000093    Male
1000094    Male
1000095    Male
1000096    Male
1000097    Male
Name: Gender, dtype: object


Select Metrics

In [15]:
df['Risk'] = df['TotalClaims'] / df['TotalPremium']

Data Segementation

In [16]:
# Group by Province
grouped_by_province = df.groupby('Province')['Risk'].mean().reset_index()
#Group by PostalCode 
grouped_by_postalcode = df.groupby('PostalCode')['Risk'].mean().reset_index() 
# Grouped by gender 
grouped_by_gender = df.groupby('Gender')['Risk'].mean().reset_index()


Statistical Testing

In [17]:
#Chi-squared test for categorical data(Province, PostalCode)
contingency_table_province = pd.crosstab(df['Province'], df['Risk'])
chi2, p_province, dof, ex = chi2_contingency(contingency_table_province)

contingency_table_postal = pd.crosstab(df['PostalCode'], df['Risk'])
chi2, p_zip, dof, ex = chi2_contingency(contingency_table_postal)

# T-test for numerical data(Gender)
risk_men = df[df['Gender'] == 'Male']['Risk']
risk_women = df[df['Gender'] == 'Female']['Risk']
t_stat, p_gender = ttest_ind(risk_men, risk_women)

In [19]:
# Analyze the p-value
alpha = 0.05

# Province
if p_province < alpha:
    province_result = "Reject the null hypothesis: There are risk differences across provinces."
else:
    province_result = "Fail to reject the null hypothesis: There are no risk differences across provinces."

# Zip Code
if p_zip < alpha:
    zip_result = "Reject the null hypothesis: There are risk differences between zip codes."
else:
    zip_result = "Fail to reject the null hypothesis: There are no risk differences between zip codes."

# Gender
if p_gender < alpha:
    gender_result = "Reject the null hypothesis: There are significant risk differences between Women and Men."
else:
    gender_result = "Fail to reject the null hypothesis: There are no significant risk differences between Women and Men."

# Print results
print(province_result)
print(zip_result)
print(gender_result)

Fail to reject the null hypothesis: There are no risk differences across provinces.
Fail to reject the null hypothesis: There are no risk differences between zip codes.
Fail to reject the null hypothesis: There are no significant risk differences between Women and Men.


DOcumentation of Analysis by Python code

In [21]:
# Document findings
report = {
    "Province Analysis": {
        "p-value": p_province,
        "Conclusion": province_result,
        "Impact": "If significant, adjust risk assessment and pricing strategies based on geographic location."
    },
    "Zip Code Analysis": {
        "p-value": p_zip,
        "Conclusion": zip_result,
        "Impact": "If significant, consider more granular pricing models or targeted marketing strategies."
    },
    "Gender Analysis": {
        "p-value": p_gender,
        "Conclusion": gender_result,
        "Impact": "If significant, review underwriting policies to ensure fairness and compliance."
    }
}

# Print the report
for key, value in report.items():
    print(f"{key}:")
    for sub_key, sub_value in value.items():
        print(f"  {sub_key}: {sub_value}")

Province Analysis:
  p-value: 1.0
  Conclusion: Fail to reject the null hypothesis: There are no risk differences across provinces.
  Impact: If significant, adjust risk assessment and pricing strategies based on geographic location.
Zip Code Analysis:
  p-value: 1.0
  Conclusion: Fail to reject the null hypothesis: There are no risk differences between zip codes.
  Impact: If significant, consider more granular pricing models or targeted marketing strategies.
Gender Analysis:
  p-value: nan
  Conclusion: Fail to reject the null hypothesis: There are no significant risk differences between Women and Men.
  Impact: If significant, review underwriting policies to ensure fairness and compliance.
